<a href="https://colab.research.google.com/github/chethanmo/chethanmo.github.io/blob/master/ath.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas pandas-ta numpy matplotlib yfinance
!pip install --upgrade pandas-datareader

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 KB 11.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 KB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 108.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 KB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 KB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 KB 15.3 MB/s eta 0:00:00
  Created wheel for pandas-ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218923 sha256=175eb9fdf7831adc7f26a8b967e6e26db10cc533159e0f7d1e6271f9af0370c9
  Stored in directory: /root/.cache/pip/wheels/54/4a/75/06b8e63fce6f6d2c1baae5c208edb18eca128407b0c96e1153
Successfully built pandas-ta


In [2]:
import pandas_ta as ta
import pandas as pd
import matplotlib.pyplot as plt
import math as math
import yfinance as yf
from dateutil.relativedelta import relativedelta
from datetime import date as date


In [3]:
#ticker_yahoo = yf.Ticker('HINDZINC.NS')
# ticker_yahoo.fast_info

file_path = '/content/EQUITY_L.csv'
fields = ['SYMBOL']
tickers = pd.read_csv(file_path, usecols=fields)


In [4]:
##lazy-loading dict with keys = ['currency', 'exchange', 'timezone', 'shares', 'market_cap', 'last_price', 'previous_close', 'open', 'day_high', 'day_low', 'regular_market_previous_close', 'last_volume', 'fifty_day_average', 'two_hundred_day_average', 'ten_day_average_volume', 'three_month_average_volume', 'year_high', 'year_low', 'year_change']

In [5]:
from concurrent.futures import ThreadPoolExecutor

ath_list = []
for i in range(0,len(tickers)):
  ath_list.append(tickers['SYMBOL'][i] +'.NS')
  


def get_stats(ticker):
    info = yf.Tickers(ticker).tickers[ticker].fast_info
    #print(f"{ticker} {info['last_price']} {info['year_high']}")
    return ticker, info['last_price'], info['year_high'], info['two_hundred_day_average']
def paralel_func(ticker):
    with ThreadPoolExecutor(max_workers=None) as executor:
        results = list(executor.map(get_stats, ath_list))
    return results

results = paralel_func(ath_list)


In [6]:
ticker_list = pd.DataFrame(results, columns=['Ticker', 'Last_price', 'ATH', 'two_EMA']) #creating the dataframe

pd.set_option('display.max_rows', None)

ticker_list = ticker_list.query('Last_price < ATH * 0.95 and Last_price > ATH * 0.9 and Last_price > two_EMA')
ticker_list.reset_index(drop=True, inplace=True)

#ticker_list.to_csv('Watchlist.csv', index=False)
#ticker_list


In [7]:
startdate = date.today() - relativedelta(years=4)
enddate = date.today()

x = ticker_list['Ticker'].to_string(header=False,index=False).split('\n')
tickers = [','.join(ele.split()) for ele in x]

df = yf.download(tickers, start= startdate, end= enddate)


[*********************100%***********************]  109 of 109 completed


In [12]:
pd.set_option('display.max_rows', 10)

ax_df = df.copy()

df1 = pd.DataFrame()


for ticker in tickers:
  test = ta.supertrend(close=ax_df[('Close', ticker)], high=ax_df[('High', ticker)], low=ax_df[('Low', ticker)], length=10, multiplier=1)
  test1 = ta.supertrend(close=ax_df[('Close', ticker)], high=ax_df[('High', ticker)], low=ax_df[('Low', ticker)], length=11, multiplier=2)
  test2 = ta.supertrend(close=ax_df[('Close', ticker)], high=ax_df[('High', ticker)], low=ax_df[('Low', ticker)], length=12, multiplier=3)
  test3 = ta.ema(close=ax_df[('High', ticker)], length=20).to_frame('EMA200_H')
  test4 = ta.ema(close=ax_df[('Close', ticker)], length=20).to_frame('EMA200_C')
  test5 = ta.ema(close=ax_df[('Low', ticker)], length=20).to_frame('EMA200_L')
  test.columns = pd.MultiIndex.from_product([test.columns, [ticker]])
  test1.columns = pd.MultiIndex.from_product([test1.columns, [ticker]])
  test2.columns = pd.MultiIndex.from_product([test2.columns, [ticker]])
  test3.columns = pd.MultiIndex.from_product([test3.columns, [ticker]])
  test4.columns = pd.MultiIndex.from_product([test4.columns, [ticker]])
  test5.columns = pd.MultiIndex.from_product([test5.columns, [ticker]])
  df1 = pd.concat([df1, test, test1, test2, test3, test4, test5], axis=1)
  df1.sort_index()
  
ax_df = pd.concat([ax_df, df1], axis=1)
ax_df.reset_index()
ax_df = ax_df.drop(['Adj Close','Open','Low','High','Volume'], level=0, axis=1)


In [13]:
ax_df.tail(2)

Close                             \
                          AARTIPHARM.NS ABBOTINDIA.NS ABCAPITAL.NS   
Date                                                                 
2023-02-03 00:00:00+05:30    259.500000      20872.25       141.75   
2023-02-06 00:00:00+05:30    272.450012      20869.75       145.75   

                                                                             \
                          AEGISCHEM.NS  AGARIND.NS      AHL.NS AKZOINDIA.NS   
Date                                                                          
2023-02-03 00:00:00+05:30   367.200012  641.250000  239.199997  2210.300049   
2023-02-06 00:00:00+05:30   358.649994  676.599976  250.600006  2202.800049   

                                                                 ...  \
                          ANANDRATHI.NS  APARINDS.NS APTECHT.NS  ...   
Date                                                             ...   
2023-02-03 00:00:00+05:30    763.150024  1931.099976      362.5  ...   
2023-02-06 00:00:00+05:30    790.349976  1928.099976      365.0  ...   

                          SUPERTd_11_2.0 SUPERTl_11_2.0 SUPERTs_11_2.0  \
                                 VSSL.NS        VSSL.NS        VSSL.NS   
Date                                                                     
2023-02-03 00:00:00+05:30             -1            NaN     346.597215   
2023-02-06 00:00:00+05:30             -1            NaN     346.597215   

                          SUPERT_12_3.0 SUPERTd_12_3.0 SUPERTl_12_3.0  \
                                VSSL.NS        VSSL.NS        VSSL.NS   
Date                                                                    
2023-02-03 00:00:00+05:30    300.470923              1     300.470923   
2023-02-06 00:00:00+05:30    300.470923              1     300.470923   

                          SUPERTs_12_3.0    EMA200_H    EMA200_C    EMA200_L  
                                 VSSL.NS     VSSL.NS     VSSL.NS     VSSL.NS  
Date                                                                          
2023-02-03 00:00:00+05:30            NaN  325.526388  316.511159  309.143437  
2023-02-06 00:00:00+05:30            NaN  326.171493  317.619619  309.510729  

[2 rows x 1744 columns]